In [4]:
import pandas as pd
import numpy as np
import datetime
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter
from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from scipy.stats import entropy

In [7]:
from catboost import CatBoostClassifier

# 載入模型路徑
model_path = 'catboost_model.cbm'

# Create an instance of the CatBoostClassifier
loaded_model = CatBoostClassifier()

# Load the model from the file
loaded_model.load_model(model_path, format='cbm')


In [8]:
new_val_data = pd.read_parquet('val_data.parquet')
example = pd.read_csv('dataset_2nd/private_2_template_v2.csv')
new_val_data = new_val_data.set_index('txkey')

In [9]:
columns_to_drop = ['chid', 'cano', 'bnsfg', 'flbmk', 'ovrlt', 'iterm', 'first_use_date',
                   'last_use_date', 'days_active']

cat_features = ['contp', 'etymd', 'mcc', 'ecfg', 'stocn', 'scity', 'insfg', 'mchno', 'acqic',
                'stscd', 'hcefg', 'csmcu', 'flg_3dsmk', 'hour']

X = new_val_data.drop(columns=columns_to_drop)

for feature in cat_features:
    X[feature] = X[feature].astype(str)
    
test_pool = Pool(X, cat_features=cat_features)


In [11]:
best_threshold = 0.7551667421788241
y_pred_proba = loaded_model.predict_proba(test_pool)[:, 1]
y_pred_custom = (y_pred_proba > best_threshold).astype(int)
new_val_data['pred'] = y_pred_custom
new_val_data = new_val_data.reset_index()
output_df = new_val_data[['txkey', 'pred']].set_index('txkey')
example = example.drop_duplicates(subset='txkey')

df2_sorted = example[['txkey']].merge(output_df, on='txkey', how='left')
df2_sorted = df2_sorted.set_index('txkey')
df2_sorted

,pred
txkey,
2dc3cfff76e54b508aec3a1ab7d3f3b9770d670c9cb9888f45ab96039212d609,0
5e4e0386a88cdf7847631e80fb129bbbf942a8ab68390bed581bf25738547e2f,0
ba2ec4f84fa996e4dbcb42984451b217fe2e9cd0b7d3da363c76df34b59ee6e0,0
a6f137a7444829310471ff9dd051addc349920ae07c5bbbd86ee4a58c05c8d8f,0
b4b3f47913e3142ca7cc54ab5d98fafc00174ad1c632069b3babc30f7ad9daad,0
...,...
f192c75180efd46e8746c9926d48f89b4f9bd730974d675d68f6bedc82ecdc06,0
79a032788e6f1c684a8c1ac514f8bd3c9fc9949869860492e9e5bea9c623db83,0
03bd77b8d2e0fa4a2d5d6fa417afb3f94ba65e4d3075ff494c8690ce18af3378,0


In [14]:
df2_sorted.to_csv('dataset_2nd/TEAM_4614.csv')

In [15]:
df2_sorted.value_counts()

pred
0       801700
1         4021
Name: count, dtype: int64